In [1]:
import glob

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance

from osgeo import gdal
gdal.SetConfigOption('GDAL_PDF_LAYERS', 'Map_Frame')
# gdal.SetConfigOption('GDAL_PDF_LAYERS_OFF',
#                      "Map_Frame.Projection_and_Grids")
                     
                     

In [2]:
# Coordinates of upper left corner, add -0.125N, 0.125W to get lower right
map_corners = [
    (38.375, -110.125, 'UT_Cleopatras_Chair_20200206_TM_geo.pdf'),
    (38.375, -109.625, 'UT_Eightmile_Rock_20200206_TM_geo.pdf'),
    (38.25, -110.125, 'UT_Elaterite_Basin_20200206_TM_geo.pdf'),
    (38.25, -109.625, 'UT_Harts_Point_North_20200206_TM_geo.pdf'),
    (38.375, -109.75, 'UT_Lockhart_Basin_20200206_TM_geo.pdf'),
    (38.375, -109.875, 'UT_Monument_Basin_20200206_TM_geo.pdf'),
    (38.25, -109.75, 'UT_North_Six-shooter_Peak_20200206_TM_geo.pdf'),
    (38.25, -110.0, 'UT_Spanish_Bottom_20200206_TM_geo.pdf'),
    (38.25, -109.875, 'UT_The_Loop_20200206_TM_geo.pdf'),
    (38.375,-110.0, 'UT_Turks_Head_20200206_TM_geo.pdf'),
]

In [3]:
def get_corners_from_gdal_ds(gdal_ds):
    ulx, xres, _, lry, _, yres = gdal_ds.GetGeoTransform()
    uly = orig_pdf.RasterYSize * yres + lry
    lrx = orig_pdf.RasterXSize * xres + ulx
    return ulx, uly, lrx, lry

# Get approx aspect ratio - just pick random one from the middle
fn = map_corners[len(map_corners)//2][-1]
orig_pdf = gdal.Open(fn)
ulx, uly, lrx, lry = get_corners_from_gdal_ds(orig_pdf)
ASPECT_RATIO = (ulx - lrx) / (uly - lry)

def warp_pdf(map_info):
    uly, ulx, quadrangle = map_info
    orig_pdf = gdal.Open(quadrangle)
    target_size = 4096 # equidistant
    warped = gdal.Warp(
        'warped.tif', orig_pdf, 
        dstSRS='EPSG:4326', # WGS84 
        resampleAlg='cubic',
        width=target_size, height=target_size,
        # this takes min_x, min_y, max_x, max_y
        outputBounds=(ulx, uly - 0.125, ulx + 0.125, uly)
    );
    warped.FlushCache();
    return warped

def gdal_ds_to_image(gdal_ds):
    a1 = gdal_ds.GetRasterBand(1).ReadAsArray()
    a2 = gdal_ds.GetRasterBand(2).ReadAsArray()
    a3 = gdal_ds.GetRasterBand(3).ReadAsArray()

    arr = np.stack((a1, a2, a3), axis=2)
    return Image.fromarray(arr)

def correct_aspect_ratio(im):
    return im.resize(
        (int(im.width * ASPECT_RATIO), im.height), Image.Resampling.BICUBIC
    )

def make_name_readable(fn):
    return fn.split('_20')[0][3:]
    

def do_all_the_things(map_info):
    im = correct_aspect_ratio(gdal_ds_to_image(warp_pdf(map_info)))
    im.save(f'{make_name_readable(map_info[-1])}.tif')
    return im


In [4]:
def merge_files(map_corners, crop_pixels=(0, 0, 0, 0)):
    
    row_lats = list(set(m[0] for m in map_corners))
    row_lats.sort()
    column_lons = list(set(m[1] for m in map_corners))
    column_lons.sort()

    # n.b. all input files have been transformed to the same size
    for ir, row in enumerate(row_lats):
        print(row)
        maps = [m for m in map_corners if m[0] == row]
        print(', '.join(make_name_readable(m[-1]) for m in maps))

        for ic, col in enumerate(column_lons):
            m = [m for m in maps if m[1] == col][0][-1]
            if ic == 0:
                im = Image.open(f'{make_name_readable(m)}.tif')
                im = crop_image(im, crop_pixels)
                continue 
            im_new = Image.open(f'{make_name_readable(m)}.tif')
            im_new = crop_image(im_new, crop_pixels)
            im = merge_im_horizontally(im, im_new)
        
        if ir == 0:
            im_all = im
            continue
        im_all = merge_im_vertically(im_all, im)

    return im_all

def crop_image(im, n_pixels):
    box = (n_pixels[0], n_pixels[1], im.width - n_pixels[2], im.height - n_pixels[3])
    return im.crop(box)
            
def merge_im_horizontally(im1, im2):
    w = im1.width + im2.width 
    h = im1.height
    im = Image.new("RGB", (w, h))
    im.paste(im1)
    im.paste(im2, (im1.width, 0))
    return im

def merge_im_vertically(im1, im2):
    w = im1.width
    h = im1.height + im2.height 
    im = Image.new("RGB", (w, h))
    im.paste(im2)
    im.paste(im1, (0, im2.height))
    return im

def increase_contrast(im, contrast_scale=3):
    enhancer = ImageEnhance.Contrast(im)
    return enhancer.enhance(contrast_scale)

In [5]:
for m in map_corners:
    # if len(glob.g÷lob(f'{make_name_readable(m[-1])}.tif')) == 0: # skip files that have already been processed
    print(make_name_readable(m[-1]))
    im = do_all_the_things(m)
    # break


im = merge_files(map_corners, (2, 1, 1, 2))
im.save('test.tif')
im3 = increase_contrast(im, 1.5)
sc = 1.5
im2 = im3.resize((int(im.width / sc), int(im.height / sc)), Image.Resampling.BICUBIC)
im2.save('final_map.tif')

Cleopatras_Chair


ERROR 1: Pos = 62570527, insufficient arguments for Marked Content
ERROR 1: Pos = 62570527, insufficient arguments for Marked Content
ERROR 1: Pos = 62570534, insufficient arguments for Marked Content
ERROR 1: Pos = 62570534, insufficient arguments for Marked Content
ERROR 1: Pos = 62570552, insufficient arguments for Marked Content
ERROR 1: Pos = 62570559, insufficient arguments for Marked Content
ERROR 1: Pos = 62570566, insufficient arguments for Marked Content
ERROR 1: Pos = 62570566, insufficient arguments for Marked Content
ERROR 1: Pos = 62570569, insufficient arguments for Marked Content
ERROR 1: Pos = 62570569, insufficient arguments for Marked Content
ERROR 1: Pos = 62570569, insufficient arguments for Marked Content
ERROR 1: Pos = 62570569, insufficient arguments for Marked Content
ERROR 1: Pos = 62570576, insufficient arguments for Marked Content
ERROR 1: Pos = 62570580, insufficient arguments for Marked Content
ERROR 1: Pos = 62572463, insufficient arguments for Marked Con

Eightmile_Rock


ERROR 1: Pos = 43700946, insufficient arguments for Marked Content
ERROR 1: Pos = 43700952, insufficient arguments for Marked Content
ERROR 1: Pos = 43700954, insufficient arguments for Marked Content
ERROR 1: Pos = 43700954, insufficient arguments for Marked Content
ERROR 1: Pos = 43700975, insufficient arguments for Marked Content
ERROR 1: Pos = 43700982, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700984, insufficient arguments for Marked Content
ERROR 1: Pos = 43700992, insufficient arguments for Marked Content
ERROR 1: Pos = 43700996, insufficient arguments for Marked Con

Elaterite_Basin


...

North_Six-shooter_Peak


ERROR 1: Pos = 46921869, insufficient arguments for Marked Content
ERROR 1: Pos = 46921869, insufficient arguments for Marked Content
ERROR 1: Pos = 46921874, insufficient arguments for Marked Content
ERROR 1: Pos = 46921889, insufficient arguments for Marked Content
ERROR 1: Pos = 46921897, insufficient arguments for Marked Content
ERROR 1: Pos = 46921903, insufficient arguments for Marked Content
ERROR 1: Pos = 46921903, insufficient arguments for Marked Content
ERROR 1: Pos = 46921905, insufficient arguments for Marked Content
ERROR 1: Pos = 46921905, insufficient arguments for Marked Content
ERROR 1: Pos = 46921905, insufficient arguments for Marked Content
ERROR 1: Pos = 46921905, insufficient arguments for Marked Content
ERROR 1: Pos = 46921913, insufficient arguments for Marked Content
ERROR 1: Pos = 46921917, insufficient arguments for Marked Content
ERROR 1: Pos = 46923809, insufficient arguments for Marked Content
ERROR 1: Pos = 46923814, insufficient arguments for Marked Con

Spanish_Bottom


ERROR 1: Pos = 86033405, insufficient arguments for Marked Content
ERROR 1: Pos = 86033411, insufficient arguments for Marked Content
ERROR 1: Pos = 86033413, insufficient arguments for Marked Content
ERROR 1: Pos = 86033429, insufficient arguments for Marked Content
ERROR 1: Pos = 86033429, insufficient arguments for Marked Content
ERROR 1: Pos = 86033438, insufficient arguments for Marked Content
ERROR 1: Pos = 86033440, insufficient arguments for Marked Content
ERROR 1: Pos = 86033443, insufficient arguments for Marked Content
ERROR 1: Pos = 86033443, insufficient arguments for Marked Content
ERROR 1: Pos = 86033443, insufficient arguments for Marked Content
ERROR 1: Pos = 86033443, insufficient arguments for Marked Content
ERROR 1: Pos = 86033443, insufficient arguments for Marked Content
ERROR 1: Pos = 86033451, insufficient arguments for Marked Content
ERROR 1: Pos = 86033455, insufficient arguments for Marked Content
ERROR 1: Pos = 86035346, insufficient arguments for Marked Con

The_Loop
Turks_Head
38.25
Elaterite_Basin, Harts_Point_North, North_Six-shooter_Peak, Spanish_Bottom, The_Loop
38.375
Cleopatras_Chair, Eightmile_Rock, Lockhart_Basin, Monument_Basin, Turks_Head
